### 使用新的方法对图像有效区域进行计算(去黑边)

In [1]:
import logging
import sys, os
import time
import glob
import numpy as np
import csv
import json
import functools
from collections import deque
import shutil
from PIL import Image

In [5]:
imagefile = r"F:\workspace\empty_scan_test\testSet5.0\1.3.6.1.4.1.52026.48447242.6617.5.1.1.dcm_frms\frm-0001.png"
im = Image.open(imagefile)
img = np.array(im)


In [19]:
print("numpy", img.shape, img.size, img[0][100])
print(im.mode)

numpy (1024, 992, 3) 3047424 [225 225 225]
RGB


In [20]:
wh = img.sum(axis=2)
print(wh.shape, wh.size, wh[0][100])

(1024, 992) 1015808 675


In [34]:
wh2 = wh.transpose()
for col in range(wh.shape[-1]):
    print(col)
    column = wh2[col]
    print(column.shape, column[0:10], column[-20:])
    sorted_col = column.sort()
    print(column.shape, column[0:10], column[-20:])
    top20 = column[-20:]
    print(type(top20), top20.mean())
    if top20.mean() > 10:
        left = col
        break
    
    

0
(1024,) [0 0 0 0 0 0 0 0 0 0] [3 3 3 3 3 3 3 3 3 3 3 3 3 3 6 6 6 6 9 9]
(1024,) [0 0 0 0 0 0 0 0 0 0] [3 3 3 3 3 3 3 3 3 3 3 3 3 3 6 6 6 6 9 9]
<class 'numpy.ndarray'> 4.2
1
(1024,) [0 0 0 0 0 0 0 0 0 0] [0 0 0 0 0 0 0 0 3 3 3 3 3 3 6 6 6 6 6 9]
(1024,) [0 0 0 0 0 0 0 0 0 0] [0 0 0 0 0 0 0 0 3 3 3 3 3 3 6 6 6 6 6 9]
<class 'numpy.ndarray'> 2.85
2
(1024,) [0 0 0 0 0 0 0 0 0 0] [ 3  3  3  3  3  3  3  3  3  3  3  6  6  6  6  6  9  9  9 12]
(1024,) [0 0 0 0 0 0 0 0 0 0] [ 3  3  3  3  3  3  3  3  3  3  3  6  6  6  6  6  9  9  9 12]
<class 'numpy.ndarray'> 5.1
3
(1024,) [0 0 0 0 0 0 0 0 0 0] [3 3 3 3 3 3 3 3 3 3 3 3 3 6 6 6 6 9 9 9]
(1024,) [0 0 0 0 0 0 0 0 0 0] [3 3 3 3 3 3 3 3 3 3 3 3 3 6 6 6 6 9 9 9]
<class 'numpy.ndarray'> 4.5
4
(1024,) [0 0 0 0 0 0 0 0 0 0] [ 66  66  75  81  81  90  90  99  99 102 105 111 120 126 138 141 150 186
 219 234]
(1024,) [0 0 0 0 0 0 0 0 0 0] [ 66  66  75  81  81  90  90  99  99 102 105 111 120 126 138 141 150 186
 219 234]
<class 'numpy.ndarray'> 118.95


In [38]:
for col in range(wh.shape[-1]-1, -1, -1):
    print(col, wh2.shape)
    column = wh2[col]
    print(column.shape, column[0:10], column[-20:])
    sorted_col = column.sort()
    print(column.shape, column[0:10], column[-20:])
    top20 = column[-20:]
    print(type(top20), top20.mean())
    if top20.mean() > 10:
        right = col
        break

991 (992, 1024)
(1024,) [0 0 0 0 0 0 0 0 0 0] [3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 6 6 6 6 6]
(1024,) [0 0 0 0 0 0 0 0 0 0] [3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 6 6 6 6 6]
<class 'numpy.ndarray'> 3.75
990 (992, 1024)
(1024,) [0 0 0 0 0 0 0 0 0 0] [3 3 3 3 3 3 6 6 6 6 6 6 6 6 6 6 6 9 9 9]
(1024,) [0 0 0 0 0 0 0 0 0 0] [3 3 3 3 3 3 6 6 6 6 6 6 6 6 6 6 6 9 9 9]
<class 'numpy.ndarray'> 5.55
989 (992, 1024)
(1024,) [0 0 0 0 0 0 0 0 0 0] [ 0  0  0  0  3  3  3  3  3  3  3  3  3  6  6  6  6  9  9 12]
(1024,) [0 0 0 0 0 0 0 0 0 0] [ 0  0  0  0  3  3  3  3  3  3  3  3  3  6  6  6  6  9  9 12]
<class 'numpy.ndarray'> 4.05
988 (992, 1024)
(1024,) [0 0 0 0 0 0 0 0 0 0] [3 3 3 3 3 3 3 3 3 3 6 6 6 6 6 6 9 9 9 9]
(1024,) [0 0 0 0 0 0 0 0 0 0] [3 3 3 3 3 3 3 3 3 3 6 6 6 6 6 6 9 9 9 9]
<class 'numpy.ndarray'> 5.1
987 (992, 1024)
(1024,) [0 0 0 0 0 0 0 0 0 0] [ 93  99 102 105 123 129 132 135 144 150 150 165 165 168 177 180 186 213
 240 261]
(1024,) [0 0 0 0 0 0 0 0 0 0] [ 93  99 102 105 123 129 132 135 144 150 150 1

In [39]:
print(left, right)

4 987
